In [5]:
from owlready2 import owlready2, get_ontology, Thing, Not, And, default_world, sync_reasoner_pellet, AllDifferent, AllDisjoint

In [6]:
owlready2.JAVA_EXE='/usr/bin/java'

In [7]:
onto = get_ontology('http://students.org/') # new ontology, providing its IRI
with onto:
    # Define a Person
    class Person(Thing):  # new subclass of Thing
        pass
    
    # Define a Course
    class Course(Thing):
        # subclass of not a Person
        is_a = [Not(onto.Person)]
        
    # AllDisjoint([Person, Course])
    
    # A Person can teach a Course
    class teaches(Person >> Course): pass
    
    # Define a Teacher
    class Teacher(Thing):
        # as a Person who teaches some Course
        equivalent_to = [And([Person, teaches.some(Course)])]
        
    # A Person can attend a Course    
    class attends(Person >> Course): pass
    
    # Define a Student
    class Student(Thing):
        # as a Person who attends some Course
        equivalent_to = [And([Person, attends.some(Course)])]

In [4]:
with onto:
    print(Person.ancestors()) # Print every superclasses
    

{students.org.Person, owl.Thing}


In [5]:
# Define some individuals
with onto:
    mary = Person(name='mary') # mary is instance of class Personne
    cs600 = Course(name='cs600')
    alice = Person(name='alice')
    alice.teaches = [cs600]
    mary.attends = [cs600]

In [6]:
 # Check what are alice and mary
with onto:
    print(alice, alice.is_a)
    print(mary, mary.is_a)

students.org.alice [students.org.Person]
students.org.mary [students.org.Person]


In [8]:
with onto:
   print("Each instances of class Person :")
   for p in Person.instances(): print(p)

Each instances of class Person :


In [8]:
with onto:
    for prop in mary.get_properties():   # Each properties of Mary with their values
        for value in prop[onto.mary]:
            print(".%s == %s" % (prop.name, value))


.attends == students.org.cs600


In [9]:
onto.save("student.owl") # This save is bug-prone - generation of <stud:Person rdf:about="alice">

In [10]:
default_world.save('world.owl') # generation of <stud:Person rdf:about="http://students.org/alice">

In [11]:
sync_reasoner_pellet([onto], infer_property_values=True, infer_data_property_values=True, debug=True, keep_tmp_file=True)

* Owlready2 * Running Pellet...
    /usr/bin/java -Xmx2000M -cp /Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/httpclient-4.2.3.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/aterm-java-1.6.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/slf4j-api-1.6.4.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/jena-iri-0.9.5.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/log4j-1.2.16.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet

In [12]:
with onto:
    # Check what are alice and mary after Pellet
    print(alice, alice.is_a)
    print(mary, mary.is_a)


students.org.alice [students.org.Teacher]
students.org.mary [students.org.Student]


In [13]:
with onto:
    # Define a new Course
    cs101 = Course(name='cs101')
    
    # Define bob as a Person
    bob = Person(name='bob')
    # who is teaching cs101
    bob.teaches = [cs101]
    # and attending cs600
    bob.attends = [cs600]
    
# print what is bob
print(bob, bob.is_a, cs101)

students.org.bob [students.org.Person] students.org.cs101


In [14]:
# Call Pellet again
sync_reasoner_pellet([onto], infer_property_values=True, infer_data_property_values=True, debug=True, keep_tmp_file=True)

* Owlready2 * Running Pellet...
    /usr/bin/java -Xmx2000M -cp /Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/httpclient-4.2.3.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/aterm-java-1.6.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/slf4j-api-1.6.4.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/jena-iri-0.9.5.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/log4j-1.2.16.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet

In [15]:
# Check what is bob
bob.is_a

[students.org.Student, students.org.Teacher]

In [16]:
# Check if any inconsistent classes
print(list(onto.world.inconsistent_classes()))

[]


In [17]:
# Add some new relation
with onto:
    class Man(Thing):
        is_a = [Person]
    class Woman(Thing):
        is_a = [Person]
        
    # Not transgender friendly
    AllDisjoint([Man,Woman])        
    
    class has_sibling(Person >> Person): pass
    
    bob.is_a.append(Man)
    alice.is_a.append(Woman)
    mary.is_a.append(Woman)

In [18]:
sync_reasoner_pellet([onto], infer_property_values=True, infer_data_property_values=True, debug=True, keep_tmp_file=True)

* Owlready2 * Running Pellet...
    /usr/bin/java -Xmx2000M -cp /Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/httpclient-4.2.3.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/aterm-java-1.6.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/slf4j-api-1.6.4.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/jena-iri-0.9.5.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/log4j-1.2.16.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet

In [19]:
bob.is_a

[students.org.Student, students.org.Teacher, students.org.Man]

In [20]:
with onto:
    imp = Imp()
    imp.set_as_rule("Person(?x), Person(?y), Person(?z), \
                     DifferentFrom(?x, ?y), DifferentFrom(?y, ?z), DifferentFrom(?x, ?z), \
                     has_sibling(?x, ?y), has_sibling(?y, ?z) -> has_sibling(?x, ?z)")

In [21]:
# Ensure that alice, bob, mary are not the same
AllDifferent([bob, alice, mary])

AllDisjoint([students.org.bob, students.org.alice, students.org.mary])

In [22]:
# Add some siblings
with onto:
    bob.has_sibling = [alice]
    alice.has_sibling = [mary]

In [23]:
sync_reasoner_pellet([onto], infer_property_values=True, infer_data_property_values=True, debug=True, keep_tmp_file=True)

* Owlready2 * Running Pellet...
    /usr/bin/java -Xmx2000M -cp /Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/httpclient-4.2.3.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/aterm-java-1.6.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/slf4j-api-1.6.4.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/jena-iri-0.9.5.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/log4j-1.2.16.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/Users/Yolaine/opt/anaconda3/lib/python3.8/site-packages/owlready2/pellet

* Owlready * Adding relation students.org.bob has_sibling students.org.mary


* Owlready2 * Pellet took 0.6487119197845459 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [24]:
bob.has_sibling

[students.org.alice, students.org.mary]

In [25]:
with onto:
    class has_sister(has_sibling):
        domain = [Person]
        range = [Woman]

In [26]:
with onto:
    Thing.descendants(include_self = True, only_loaded = False)

In [27]:
onto.save('students.owl')

In [28]:
default_world.ontologies

{'http://students.org/': get_ontology("http://students.org/"),
 'http://anonymous/': get_ontology("http://anonymous/"),
 'http://inferrences/': get_ontology("http://inferrences/")}

In [29]:
default_world.save('world.owl')

Native SPARQL engine (see [SPARQL Owlready2 documentation](https://owlready2.readthedocs.io/en/latest/sparql.html#native-sparql-engine))

RDFLib SPARQL engine

In [30]:
with onto:
    print(bob.iri)

http://students.org/bob


In [31]:
graph = default_world.as_rdflib_graph()
graph.bind('owl', 'http://www.w3.org/2002/07/owl#')

In [32]:
r = list(graph.query('SELECT ?classe WHERE { <http://students.org/bob> a ?classe . ?classe a owl:Class . }'))
print(r)

[(rdflib.term.URIRef('http://students.org/Person'),), (rdflib.term.URIRef('http://students.org/Teacher'),), (rdflib.term.URIRef('http://students.org/Student'),), (rdflib.term.URIRef('http://students.org/Man'),)]


In [33]:
graph.bind("student", "http://students.org/")

In [34]:
r = list(graph.query('SELECT ?classe WHERE { student:bob a ?classe . ?classe a owl:Class . }'))
print(r)

[(rdflib.term.URIRef('http://students.org/Person'),), (rdflib.term.URIRef('http://students.org/Teacher'),), (rdflib.term.URIRef('http://students.org/Student'),), (rdflib.term.URIRef('http://students.org/Man'),)]


In [35]:
onto.destroy()

In [36]:
bob.has_sibling

[students.org.alice, students.org.mary]

In [37]:
bob.iri

'http://students.org/bob'

In [38]:
with onto:
    print(bob.iri)

http://students.org/bob


In [39]:
onto.save('students.owl')

TypeError: 'NoneType' object is not subscriptable